## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5001")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5001'


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716489789467, experiment_id='0', last_update_time=1716489789467, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/23 14:45:27 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/1/7d9b20c82b7746bb8e4166dc0b2334e2/artifacts'


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1716489927607, experiment_id='1', last_update_time=1716489927607, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716489789467, experiment_id='0', last_update_time=1716489789467, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5001")

In [11]:
client.search_registered_models()

[]

In [20]:
run_infos = client.search_runs(experiment_ids=1)
run_info_most_recent = run_infos[0]
run_id = run_info_most_recent.info.run_id

if run_id:
    model_uri = f"runs:/{run_id}/model"  # Ensure this is the correct artifact path to your model
    result = mlflow.register_model(
        model_uri=model_uri,
        name='iris-classifier'
    )
    print("Model registered successfully:", result)
else:
    print("No runs found for the experiment.")

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/05/23 14:51:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1


Model registered successfully: <ModelVersion: aliases=[], creation_timestamp=1716490286376, current_stage='None', description='', last_updated_timestamp=1716490286376, name='iris-classifier', run_id='7d9b20c82b7746bb8e4166dc0b2334e2', run_link='', source='mlflow-artifacts:/1/7d9b20c82b7746bb8e4166dc0b2334e2/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


Created version '1' of model 'iris-classifier'.
